# 단어의 표현


### 단어의 표현이란 ? : 자연어(문자열)를 숫자로 표현하여 컴퓨터로 연산할 수 있도록 하는 것

 - Local(Discrete) vs Distributed(Conitinuous)
 
         Local(Discrete) : 표현하고자 하는 단어 하나만을 보고 각 단어에 숫자를 부여 - Onehot,Ngram,Countbased(BOW,TDM) 
     
         Distributed(Conitinuous) - 딥러닝에서 사용 : 주변 단어를 함께 고려하여 수치를 표현 - Prediction based(Word2Vec,Fasttext),   Countbased(windows-Glove, Full docunment-LSA) 
         
         
         
         
         

In [5]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

### 원핫벡터

- 차원의 문제 : 표현하고자 하는 단어가 많아지는 만큼 벡터의 크기가 커짐
- 벡터가 단어의 의미를 담지 못한다.

In [3]:
word_list=["바나나","원숭이","코끼리","사과"]

In [17]:
#Label Encoder
# 일괄적인 숫자 값으로 변환되면서 예측 성능이 떨어질 수 있다.
#-> 숫자의 크고 작음에 대한 특성이 작용
#2. 선형 회귀와 같은 ML 알고리즘에는 적용하지 않아야 함 (트리 계열의 ML알고리즘은 숫자의 이러한 특성을 반영하지 않으므로 괜찮음)




encoder = LabelEncoder()
encoder.fit(word_list)
labled_words=encoder.transform(word_list)
labled_words

array([0, 2, 3, 1], dtype=int64)

In [29]:
#Onehot encoding
#숫자로 레이블링 후에 수행


label_enc=LabelEncoder()
label_enc.fit(word_list)
labled_words=label_enc.transform(word_list)
labled_words=labled_words.reshape(-1,1) #  1열로 reshape 

onehot_enc=OneHotEncoder(sparse=False)
onehot_enc.fit(labled_words)
encoded=onehot_enc.transform(labled_words)
print(encoded)

[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]


### BoW

- 각 단어를 인덱스로 표현하고, 인덱스에 해당하는 문자의 출현 횟수를 기록

- 단어의 순서를 고려하지 않는다.
- BoW는 sparse하다.즉 벡터 공간의 낭비, 연산 비효율성을 초래한다.
- 단어 빈도수가 중요도를 바로 의미 하지 않는다.
- 전처리가 매우 중요하다. 같은 의미의 다른 단어 표현이 다른 단어로 인식될 수 있음. (뉴스와 같은 정제된 어휘를 사용하는 매체에서는 좋으나, 소셜미디어에서는 활용이 어려움)

In [57]:
docs=["오늘 동물원에서 코끼리 원숭이를 보고 코끼리 원숭이에게 먹이를 줬어","오늘 동물원에서 원숭이에게 사과를 줬어"]

In [60]:
doc_ls=[]
for doc in docs:
    doc_ls.append(doc.split())
doc_ls

[['오늘', '동물원에서', '코끼리', '원숭이를', '보고', '코끼리', '원숭이에게', '먹이를', '줬어'],
 ['오늘', '동물원에서', '원숭이에게', '사과를', '줬어']]

In [61]:
from collections import defaultdict

word2id=defaultdict(lambda:len(word2id)) #
for doc in doc_ls:
    for token in doc:
        word2id[token]
word2id

defaultdict(<function __main__.<lambda>()>,
            {'오늘': 0,
             '동물원에서': 1,
             '코끼리': 2,
             '원숭이를': 3,
             '보고': 4,
             '원숭이에게': 5,
             '먹이를': 6,
             '줬어': 7,
             '사과를': 8})

In [62]:
BoW_ls=[]
for i,doc in enumerate(doc_ls):
    bow=np.zeros(len(word2id),dtype=int)
    for token in doc:
        bow[word2id[token]]+=1
    BoW_ls.append(bow.tolist())
BoW_ls

[[1, 1, 2, 1, 1, 1, 1, 1, 0], [1, 1, 0, 0, 0, 1, 0, 1, 1]]

In [63]:
from IPython.core import display as ICD
import pandas as pd
sorted_vocab=sorted((value,key) for key,value in word2id.items())
print('sorted_vocab',sorted_vocab)

vocab=[]
for v in sorted_vocab:
    vocab.append(v[1])
print('vocab',vocab)
for i in range(len(docs)):
    print("문서{} : {}".format(i,docs[i]))
    ICD.display(pd.DataFrame([BoW_ls[i]],columns=vocab))
    print("\n\n")

sorted_vocab [(0, '오늘'), (1, '동물원에서'), (2, '코끼리'), (3, '원숭이를'), (4, '보고'), (5, '원숭이에게'), (6, '먹이를'), (7, '줬어'), (8, '사과를')]
vocab ['오늘', '동물원에서', '코끼리', '원숭이를', '보고', '원숭이에게', '먹이를', '줬어', '사과를']
문서0 : 오늘 동물원에서 코끼리 원숭이를 보고 코끼리 원숭이에게 먹이를 줬어


,오늘,동물원에서,코끼리,원숭이를,보고,원숭이에게,먹이를,줬어,사과를
0,1,1,2,1,1,1,1,1,0





문서1 : 오늘 동물원에서 원숭이에게 사과를 줬어


,오늘,동물원에서,코끼리,원숭이를,보고,원숭이에게,먹이를,줬어,사과를
0,1,1,0,0,0,1,0,1,1


#### 단어의 등장 순서를 고려하지 않는 BoW

In [69]:
docs=["나는 양념 치킨을 좋아해 하지만 후라이드 치킨을 싫어해","나는 후라이드 치킨을 좋아해 하지만 양념 치킨을 싫어해"]

In [70]:
doc_ls=[]
for doc in docs:
    doc_ls.append(doc.split())
doc_ls

[['나는', '양념', '치킨을', '좋아해', '하지만', '후라이드', '치킨을', '싫어해'],
 ['나는', '후라이드', '치킨을', '좋아해', '하지만', '양념', '치킨을', '싫어해']]

In [71]:
word2id=defaultdict(lambda : len(word2id))
for doc in doc_ls:
    for token in doc:
        word2id[token]
        
word2id

defaultdict(<function __main__.<lambda>()>,
            {'나는': 0,
             '양념': 1,
             '치킨을': 2,
             '좋아해': 3,
             '하지만': 4,
             '후라이드': 5,
             '싫어해': 6})

In [72]:
#서로 다른 문장이지만 동일한 벡터를 만들어 낸다. (단어의 등장 순서를 고려하지 않기 때문이다.)

BoW_ls=[]
for i,doc in enumerate(doc_ls):
    bow=np.zeros(len(word2id),dtype=int)
    for token in doc:
        bow[word2id[token]]+=1
    BoW_ls.append(bow.tolist())
BoW_ls

[[1, 1, 2, 1, 1, 1, 1], [1, 1, 2, 1, 1, 1, 1]]

#### 사이킷런을 이용한 BoW

In [87]:
# 사이킷런 자체적으로 공백을 기준으로 토큰화하여 벡터를 만듦 (필요한 전처리 과정은 따로 해주어야함)
# ex) 꼬꼬마등의 모듈의 이용하여 형태소 토큰화를 한 뒤, 이 토큰들을 공백을 이용하여 합친 후, 사이킷런 이용하기

from sklearn.feature_extraction.text import CountVectorizer
count_vect=CountVectorizer()
BoW=count_vect.fit_transform(docs)

In [81]:
print(BoW[0])
print(BoW[1])

  (0, 0)	1
  (0, 2)	1
  (0, 4)	2
  (0, 3)	1
  (0, 5)	1
  (0, 6)	1
  (0, 1)	1
  (0, 0)	1
  (0, 2)	1
  (0, 4)	2
  (0, 3)	1
  (0, 5)	1
  (0, 6)	1
  (0, 1)	1


In [76]:
from IPython.core import display as ICD

vocab=count_vect.get_feature_names()
for i in range(len(docs)):
    print("문서 {} : {}".format(i,docs[i]))
    ICD.display(pd.DataFrame([BoW.toarray()[i]],columns=vocab))
    print("\n\n")

문서 0 : 나는 양념 치킨을 좋아해 하지만 후라이드 치킨을 싫어해


,나는,싫어해,양념,좋아해,치킨을,하지만,후라이드
0,1,1,1,1,2,1,1





문서 1 : 나는 후라이드 치킨을 좋아해 하지만 양념 치킨을 싫어해


,나는,싫어해,양념,좋아해,치킨을,하지만,후라이드
0,1,1,1,1,2,1,1


## gensim을 이용한 BoW

In [84]:
# !pip install gensim

In [85]:
from gensim import corpora
doc_ls=[]
for doc in docs:
    doc_ls.append(doc.split())

id2word=corpora.Dictionary(doc_ls)
BoW=[]
for doc in doc_ls:
    BoW.append(id2word.doc2bow(doc))

In [86]:
BoW

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1)],
 [(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1)]]